In [ ]:
# import libraries
import tensorflow as tf
import pandas as pd
from tensorflow import keras
%pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2024-01-31 14:41:07--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.005s  

2024-01-31 14:41:08 (62.9 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2024-01-31 14:41:08--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==============

In [ ]:
df_train = pd.read_csv(train_file_path, sep = "\t", header = None, names = ['y', 'x'])
df_test = pd.read_csv(test_file_path, sep = "\t", header = None, names = ['y', 'x'])

y_train = df_train['y'].astype('category').cat.codes
y_test = df_test['y'].astype('category').cat.codes
y_train[:5]

0    0
1    0
2    0
3    0
4    0
dtype: int8

In [ ]:
import nltk
import re
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
stopwords_eng = set(stopwords.words('english'))

lemmatizer = WordNetLemmatizer()

def clean_txt(txt):
  txt = re.sub(r'([^\s\w])+', ' ', txt)
  txt = txt.lower()
  return txt

X_train = df_train['x'].apply(lambda x : clean_txt(x))
X_train[:5]

0    ahhhh just woken up had a bad dream about u th...
1                             you can never do nothing
2    now u sound like manky scouse boy steve like  ...
3    mum say we wan to go then go  then she can shu...
4    never y lei  i v lazy  got wat  dat day ü send...
Name: x, dtype: object

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

#Keep top 1000 frequently occuring words
max_words = 1000

#Cut off the words after seeing 500 words in each document
max_len = 500

t = Tokenizer(num_words = max_words)
t.fit_on_texts(X_train)

In [ ]:
sequences = t.texts_to_sequences(X_train)
sequences[:5]

seqs_matrix = sequence.pad_sequences(sequences, maxlen = max_len)
seqs_matrix[:5]

array([[  0,   0,   0, ...,  47,  13,  12],
       [  0,   0,   0, ..., 279,  31, 343],
       [  0,   0,   0, ...,  43, 435, 531],
       [  0,   0,   0, ...,  25, 344,  99],
       [  0,   0,   0, ..., 200, 143,  79]], dtype=int32)

In [ ]:
from keras import models
from keras import layers

i = tf.keras.layers.Input(shape=[max_len])
x = tf.keras.layers.Embedding(max_words, 50)(i)
x = tf.keras.layers.LSTM(64)(x)

x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(1, activation='relu')(x)

model = tf.keras.models.Model(inputs=i, outputs=x)
model.compile(
    loss='binary_crossentropy',
    optimizer='RMSprop',
    metrics=['accuracy']
)
model.summary()

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Layer (type)                       ┃ Output Shape                  ┃     Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)         │ (None, 500)                   │           0 │
├────────────────────────────────────┼───────────────────────────────┼─────────────┤
│ embedding_5 (Embedding)            │ (None, 500, 50)               │      50,000 │
├────────────────────────────────────┼───────────────────────────────┼─────────────┤
│ lstm_1 (LSTM)                      │ (None, 64)                    │      29,440 │
├────────────────────────────────────┼───────────────────────────────┼─────────────┤
│ dense_2 (Dense)                    │ (None, 256)                   │      16,640 │
├────────────────────────────────────┼───────────────────────────────┼─────────────┤
│ dropout_1 (Dropout)                │ (None, 256)                   │           0 │
├────────────────────────────────────┼───────────────────────────────┼─────────────┤
│ dense_3 (Dense)                    │ (None, 1)                     │         257 │
└────────────────────────────────────┴───────────────────────────────┴─────────────┘

 Total params: 96,337 (376.32 KB)

 Trainable params: 96,337 (376.32 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
r = model.fit(seqs_matrix, y_train,
              batch_size=128, epochs=10,
              validation_split=0.2,
              callbacks=[tf.keras.callbacks.EarlyStopping(
                  monitor='val_loss', min_delta=0.0001)])

Epoch 1/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 26s 875ms/step - accuracy: 0.8764 - loss: 0.4673 - val_accuracy: 0.9821 - val_loss: 0.1708
Epoch 2/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 40s 837ms/step - accuracy: 0.9795 - loss: 0.1300 - val_accuracy: 0.9833 - val_loss: 0.1228
Epoch 3/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 25s 919ms/step - accuracy: 0.9857 - loss: 0.0816 - val_accuracy: 0.9892 - val_loss: 0.1034
Epoch 4/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 38s 791ms/step - accuracy: 0.9888 - loss: 0.0740 - val_accuracy: 0.9761 - val_loss: 0.1486


In [ ]:
def preprocessing(X):
  x = X.apply(lambda x:clean_txt(x))
  x = t.texts_to_sequences(x)
  return sequence.pad_sequences(x, maxlen = max_len)

s = model.evaluate(preprocessing(df_test['x']), y_test)

44/44 ━━━━━━━━━━━━━━━━━━━━ 4s 86ms/step - accuracy: 0.9775 - loss: 0.1566


In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  p = model.predict(preprocessing(pd.Series([pred_text])))[0]

  return (p[0], ("ham" if p<0.5 else "spam"))

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step
(0.0, 'ham')


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
You passed the challenge. Great job!
